In [ ]:
%%capture
!wget https://raw.githubusercontent.com/shitkov/news_classification/main/bert_classifier.py
!wget https://raw.githubusercontent.com/shitkov/news_classification/main/bert_dataset.py
!wget https://raw.githubusercontent.com/shitkov/news_classification/main/requirements.txt
!wget https://raw.githubusercontent.com/shitkov/news_classification/main/news.zip
!wget https://raw.githubusercontent.com/shitkov/news_classification/main/headlines_train.csv
!wget https://raw.githubusercontent.com/shitkov/news_classification/main/headlines_test.csv
!unzip '/content/news.zip' -d '/content/'
!pip install -r /content/requirements.txt

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

from bert_dataset import CustomDataset
from bert_classifier import BertClassifier

from sklearn.metrics import precision_recall_fscore_support

Get data

In [ ]:
data_train = pd.read_csv('/content/headlines_train.csv').reset_index()
data_test = pd.read_csv('/content/headlines_test.csv').reset_index()

In [ ]:
data_valid, data_test = train_test_split(data_test, test_size=0.5, random_state=42)

In [ ]:
X_train = list(data_train['title'])
y_train = list(data_train['label'])

X_valid = list(data_valid['title'])
y_valid = list(data_valid['label'])

X_test = list(data_test['title'])
y_test = list(data_test['label'])

Initialize BERT classifier

In [ ]:
%%capture
classifier = BertClassifier(
        model_path='cointegrated/rubert-tiny',
        tokenizer_path='cointegrated/rubert-tiny',
        n_classes=2,
        epochs=20,
        model_save_path='/content/bert.pt'
)

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not i

Prepare data and helpers for train and evlauation

In [ ]:
classifier.preparation(
        X_train=X_train,
        y_train=y_train,
        X_valid=X_valid,
        y_valid=y_valid
    )

Train loop

In [ ]:
classifier.train()

Epoch 1/20
Train loss 0.6681718130990277 accuracy 0.7559386973180077
Val loss 1.633183860196732 accuracy 0.5569620253164557
----------
Epoch 2/20
Train loss 0.4934766469080397 accuracy 0.8796934865900383
Val loss 2.004106754413806 accuracy 0.6075949367088608
----------
Epoch 3/20
Train loss 0.3370206245216291 accuracy 0.9222222222222223
Val loss 2.300218697928358 accuracy 0.5949367088607594
----------
Epoch 4/20
Train loss 0.246490748185953 accuracy 0.9482758620689655
Val loss 2.223523136806398 accuracy 0.6708860759493671
----------
Epoch 5/20
Train loss 0.1523768142081054 accuracy 0.9689655172413794
Val loss 2.487241657562481 accuracy 0.6582278481012658
----------
Epoch 6/20
Train loss 0.09944107382133939 accuracy 0.9796934865900383
Val loss 2.8570859882132025 accuracy 0.6455696202531646
----------
Epoch 7/20
Train loss 0.0791458348557969 accuracy 0.9839080459770115
Val loss 3.171973788136802 accuracy 0.620253164556962
----------
Epoch 8/20
Train loss 0.053518869588844324 accuracy 0.9

In [ ]:
predictions = [classifier.predict(x) for x in X_test]
precision, recall, f1score = precision_recall_fscore_support(y_test, predictions, average='macro')[:3]

print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')

precision: 0.5890957446808511, recall: 0.5860077021822849, f1score: 0.5798549556809025


Train model on all train data

In [ ]:
classifier = BertClassifier(
        model_path='cointegrated/rubert-tiny',
        tokenizer_path='cointegrated/rubert-tiny',
        n_classes=2,
        epochs=20,
        model_save_path='/content/bert.pt'
)

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not i

In [ ]:
full_data = pd.concat([data_train, data_valid, data_test])

In [ ]:
data_train, data_valid = train_test_split(full_data, test_size=0.05, random_state=42)

X_train = list(data_train['title'])
y_train = list(data_train['label'])

X_valid = list(data_valid['title'])
y_valid = list(data_valid['label'])

In [ ]:
classifier.preparation(
        X_train=X_train,
        y_train=y_train,
        X_valid=X_valid,
        y_valid=y_valid
    )

In [ ]:
classifier.train()

Epoch 1/20
Train loss 0.6644873032223313 accuracy 0.7337390642829974
Val loss 0.8917341005522758 accuracy 0.7338129496402878
----------
Epoch 2/20
Train loss 0.5631372885928603 accuracy 0.8543172308862685
Val loss 0.6861865745034136 accuracy 0.8129496402877698
----------
Epoch 3/20
Train loss 0.39269999835227204 accuracy 0.9045264359071891
Val loss 0.7136114027700386 accuracy 0.8489208633093526
----------
Epoch 4/20
Train loss 0.30401087928222076 accuracy 0.9315329022441994
Val loss 0.8176825606430481 accuracy 0.8417266187050361
----------
Epoch 5/20
Train loss 0.21466859830240187 accuracy 0.9585393685812096
Val loss 0.6703199559232287 accuracy 0.8776978417266188
----------
Epoch 6/20
Train loss 0.14354582481784414 accuracy 0.9672879421833397
Val loss 0.7135179605986716 accuracy 0.8848920863309353
----------
Epoch 7/20
Train loss 0.09363505122601332 accuracy 0.9806009889691898
Val loss 0.7432826645088686 accuracy 0.8920863309352518
----------
Epoch 8/20
Train loss 0.0692297394263705 ac

In [20]:
predictions = [classifier.predict(x) for x in X_valid]
precision, recall, f1score = precision_recall_fscore_support(y_valid, predictions, average='macro')[:3]

print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')

precision: 0.9062758906379453, recall: 0.9066127694859039, f1score: 0.9063973063973063


Prepare test data

In [ ]:
test_data = pd.read_csv('/content/test_data.csv', index_col=0)
test_data = test_data.fillna('текст')
X_test= list(test_data['title'])

In [ ]:
predictions = [classifier.predict(x) for x in X_test]

In [ ]:
test_data['prediction'] = predictions

In [ ]:
test_data.to_csv('test_data_headlines_prediction.csv')